In [ ]:
!pip install mtcnn matplotlib face_recognition

In [ ]:
import dlib

# Import

In [ ]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from google.colab.patches import cv2_imshow
import imutils
from tqdm import tqdm
from mtcnn import MTCNN
import tensorflow as tf
import concurrent
from skimage import data, img_as_float
from skimage import exposure
import face_recognition
from skimage import img_as_ubyte
from collections import Counter
import json
os.chdir('/content/drive/MyDrive/Colab Notebooks')

# 1- Read dataset Metadata
name: File name <br>
label: Fake | Real <br>
split: Is on test or training set <br>
original: If it's fake, original's file name counterpart.

In [ ]:
f = open(os.path.join('Dataset','metadata.json'), 'r')
labels = json.load(f)
real_videos = [x for x in labels.keys() if labels[x]['label'] == 'REAL']

In [ ]:
cap = cv2.VideoCapture(os.path.join('Dataset', 'bkzrsfcrcr.mp4'))
ret, frame = cap.read()

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.show()

# GET Height
print(f'Height: {cap.get(cv2.CAP_PROP_FRAME_HEIGHT)}')

# GET Width
print(f'Width: {cap.get(cv2.CAP_PROP_FRAME_WIDTH)}')

# GET nº of Frames
print(f'Nº of Frames: {cap.get(cv2.CAP_PROP_FRAME_COUNT)}')

# GET FPS
print(f'FPS: {cap.get(cv2.CAP_PROP_FPS)}')


# 2.1 - Crop only Face with Haarcascade

It created a lot of inconsistencies

In [ ]:
from matplotlib.patches import Rectangle
def create_color_hist(img):
  channels = [Counter(img[:,:,i].reshape(-1)) for i in range(3)]
  hist = {i:[channels[i][intensity] for intensity in range(256)] for i in range(3)}

  return hist

def crop_face_haar(cap, frame_skip):
  # print('COMECEI')
  fps = cap.get(cv2.CAP_PROP_FPS)
  frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  p_frames = []
  i = 0
  for frame_i in range(0, frame_count, frame_skip):
    ret, frame_original = cap.read()
    frame_original = cv2.cvtColor(frame_original, cv2.COLOR_BGR2RGB)
    frame = cv2.cvtColor(frame_original, cv2.COLOR_RGB2GRAY)
    frame = exposure.equalize_hist(frame)
    frame = img_as_ubyte(frame)
    print(f'FRAME: {frame_i}')
    if i > frame_skip:
      faces = prev_face
    else:
      faces = face_recognition.face_locations(frame, model="hog")
    # print(faces)
    if len(faces) >= 1:
      prev_face = faces
    else:
      try:
        faces = prev_face
      except:
        prev_face = None
        i = 0
        # while faces == None or (type(faces) == list and len(faces) == 0):
        while prev_face == None or len(prev_face) == 0:
          i += 1
          # print(faces)
          ret_next, frame_next = cap.read()
          frame = cv2.cvtColor(frame_next, cv2.COLOR_RGB2GRAY)
          frame = exposure.equalize_hist(frame)
          faces = face_recognition.face_locations(frame_next, model="hog")
          prev_face = faces
          if i >= 50:
            return False
    # print(faces)
    # print(prev_face)
    top, right, bottom, left = faces[0]
    vertF = 300 - (bottom - top)
    horF = 300 - (right - left)
    topF, rightF, botF, leftF = top, right, bottom, left
    if vertF != 0:
      topF = max(0, top - vertF//2)
      # print(topF)
      botF = bottom + (vertF - vertF//2) - min(0, top - vertF//2)
    if horF != 0:
      leftF = max(0, left - horF//2)
      rightF = right + (horF - (horF//2)) - min(0, left - horF//2)
    p_frames.append(frame_original[topF:botF, leftF:rightF])
    # print(len(frame_original[topF:botF, leftF:rightF]),len(frame_original[topF:botF, leftF:rightF][0]))
    # print(f'{topF}:{botF} \\\ {leftF}:{rightF}')
    # plt.figure()
    # plt.imshow(frame_original[topF:botF,leftF:rightF])
    # plt.show()
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_i-1)
    # xl, yl, wl, hl = x, y, w, h
  return p_frames

for path in glob.glob(os.path.join('Dataset', '*'))[740:]:
  print(path)
  video_name = path.split('/')[1]
  if video_name in real_videos:
    cap = cv2.VideoCapture(path)
    print(video_name)

    dimensions = (300, 300)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    video_writer = cv2.VideoWriter(os.path.join('Output', 'processed_'+video_name), cv2.VideoWriter_fourcc(*'DIVX'), fps, dimensions, isColor=True)
    # with tf.device('/device:GPU:0'):
    video_frames = crop_face_haar(cap, 10)
    # print(len(video_frames))
    # print([len(face_frame) for face_frame in video_frames])
    if video_frames != False:
      [video_writer.write(cv2.cvtColor(face_frame, cv2.COLOR_RGB2BGR)) for face_frame in video_frames]
      cap.release()
      video_writer.release()
    else:
      file1 = open("video_errors.txt", "a")  # append mode
      file1.write(f"{video_name} \n")
      file1.close()

In [ ]:
# Append-adds at last
file1 = open("myfile.txt", "a")  # append mode
file1.write("Today \n")
file1.close()

In [ ]:
open("myfile.txt", "x")

In [ ]:

video_name = 'dzdbunuaag.mp4'

cap = cv2.VideoCapture(os.path.join('Dataset', video_name))

dimensions = (300, 300)

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_writer = cv2.VideoWriter(os.path.join('Output', 'face_'+video_name), cv2.VideoWriter_fourcc(*'DIVX'), fps, dimensions, isColor=False)

# for frame_i in range(2):
ret, frame = cap.read()
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
# frame = cv2.convertScaleAbs(frame, alpha=4, beta=5)

p2, p98 = np.percentile(frame, (2, 98))
img_rescale = exposure.rescale_intensity(frame, in_range=(p2, p98))

# Equalization
img_eq = exposure.equalize_hist(frame)

# Adaptive Equalization
img_adapteq = exposure.equalize_adapthist(frame, clip_limit=0.03)

  detector = cv2.FaceDetectorYN.create("Models/face_detection_yunet.onnx",  "", (0, 0))
  detector.setInputSize((width, height))

  _, faces = detector.detect(frame)

  # if faces[1] is None, no face found

  if faces is not None:
    for face in faces:
        # parameters: x1, y1, w, h, x_re, y_re, x_le, y_le, x_nt, y_nt, x_rcm, y_rcm, x_lcm, y_lcm

        # bouding box
        box = list(map(int, face[:4]))
        color = (0, 0, 255)
        cv2.rectangle(frame, box, color, 5)

        # confidence
        confidence = face[-1]
        confidence = "{:.2f}".format(confidence)
        position = (box[0], box[1] - 10)
        cv2.putText(frame, confidence, position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 3, cv2.LINE_AA)
plt.figure()
plt.imshow(frame)
plt.figure()
plt.imshow(img_rescale)
plt.figure()
plt.imshow(img_eq)
plt.figure()
plt.imshow(img_adapteq)

In [ ]:
face_cascade = cv2.CascadeClassifier('Models/haarcascade_frontalface_alt2.xml')
frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(frame, 1.1, 4)

for (x, y, w, h) in faces:
    # cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
    faces = frame[y:y + h, x:x + w]

    # cv2_imshow(faces)
    cv2_imshow(imutils.resize(faces, 350, 350))
    # cv2.imwrite('face.jpg', faces)

In [ ]:

video_name = 'dzdbunuaag.mp4'

cap = cv2.VideoCapture(os.path.join('Dataset', video_name))

dimensions = (300, 300)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_writer = cv2.VideoWriter(os.path.join('Output', 'face_'+video_name), cv2.VideoWriter_fourcc(*'DIVX'), fps, dimensions, isColor=False)

face_cascade = cv2.CascadeClassifier('Models/haarcascade_frontalface_alt2.xml')


for frame_i in range(0,frame_count, 10):
  ret, frame = cap.read()
  frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  frame = exposure.equalize_hist(frame)
  frame = img_as_ubyte(frame)
  faces = face_cascade.detectMultiScale(frame, 1.1, 3)
  if len(faces) == 1:
    prev_face = faces
  else:
    faces = prev_face
  x, y, w, h = faces[0]
  width_fill = 300 - ((x + w) - x)
  height_fill = 300 - ((y + h) - y)
  face_frame = frame[y:y + h + height_fill, x:x + w + width_fill]

  video_writer.write(face_frame)

cap.release()
video_writer.release()

In [ ]:
def crop_face_haar(cap, frame_skip):
  fps = cap.get(cv2.CAP_PROP_FPS)
  face_cascade = cv2.CascadeClassifier('Models/haarcascade_frontalface_alt2.xml')
  frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  p_frames = []
  for frame_i in range(0, frame_count, frame_skip):
    ret, frame = cap.read()
    print(f'FRAME: {frame_i}')
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    faces = face_cascade.detectMultiScale(frame, 1.1, 4)
    # print(len(faces[0]['box']))
    print(faces)
    if len(faces) == 4:
      prev_face = faces
    else:
      try:
        faces = prev_face
      except:
        while len(faces) < 1:
          print(faces)
          ret_next, frame_next = cap.read()
          faces = face_cascade.detectMultiScale(frame, 1.1, 4)
    print(faces)
    x, y, w, h = faces[0]
    width_fill = 300 - ((x + w + ((w - wl)/2)) - x)
    height_fill = 300 - ((y + h + ((h - hl)/2)) - y)
    p_frames.append(frame[y:y + h + height_fill, x:x + w + width_fill])
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_i-1)
    xl, yl, wl, hl = x, y, w, h
  return p_frames

# video_name =
print(glob.glob(os.path.join('Dataset', '*'))[13])
video_name = glob.glob(os.path.join('Dataset', '*'))[0].split('/')[1]
cap = cv2.VideoCapture(glob.glob(os.path.join('Dataset', '*'))[13])

dimensions = (300, 300)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_writer = cv2.VideoWriter(os.path.join('Output', 'face_'+video_name), cv2.VideoWriter_fourcc(*'DIVX'), fps, dimensions, isColor=False)

# with tf.device('/device:GPU:0'):
video_frames = crop_face_haar(cap, 10)
print(video_frames)
[video_writer.write(cv2.cvtColor(face_frame, cv2.COLOR_RGB2BGR)) for face_frame in video_frames]
cap.release()
video_writer.release()

In [ ]:
print(glob.glob(os.path.join('Dataset', '*'))[13:])

# 2.2 - Crop only Faces with MTCNN

The flickering got smoother, but it's still there. <br>
Additionaly, seeking to save processing time, I've set the pre-processing to process only 6 of the 30 Frames per second, jumping from 5 to 5.

In [ ]:
def crop_face(cap, frame_skip):
  fps = cap.get(cv2.CAP_PROP_FPS)
  frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  p_frames = []
  for frame_i in range(0, frame_count, frame_skip):
    ret, frame = cap.read()
    print(f'FRAME: {frame_i}')
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = exposure.equalize_hist(frame)
    faces = f_detector.detect_faces(frame)
    # print(len(faces[0]['box']))
    print(faces)
    if len(faces) >= 1:
      prev_face = faces
    else:
      try:
        faces = prev_face
      except:
        while len(faces) < 1:
          print(faces)
          ret_next, frame_next = cap.read()
          faces = f_detector.detect_faces(frame_next)

    x, y, w, h = faces[0]['box']
    if frame_i == 0:
      xl, yl, wl, hl = x, y, w, h
    width_fill = 300 - (((x + w) - x) + ((xl + wl) - xl))//2
    height_fill = 300 - (((y + h) - y) + ((yl + hl) - yl))//2
    p_frames.append(frame[y:y + h + height_fill, x:x + w + width_fill])
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_i-1)
    xl, yl, wl, hl = x, y, w, h
  return p_frames


for file_path in glob.glob(os.path.join('Dataset', '*'))[13:]:
  print(file_path)
  video_name = file_path.split('/')[1]

  cap = cv2.VideoCapture(file_path)

  dimensions = (300, 300)
  fps = cap.get(cv2.CAP_PROP_FPS)
  frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

  fourcc = cv2.VideoWriter_fourcc(*'XVID')
  video_writer = cv2.VideoWriter(os.path.join('Output', 'mtcnn_face_'+video_name), cv2.VideoWriter_fourcc(*'DIVX'), fps, dimensions, isColor=True)

  f_detector = MTCNN()

  # with tf.device('/device:GPU:0'):
  video_frames = crop_face(cap, 10)
  [video_writer.write(cv2.cvtColor(face_frame, cv2.COLOR_RGB2BGR)) for face_frame in video_frames]
  cap.release()
  video_writer.release()





# 3.1 - Face Crop MTCNN + Canny Edge Detector.

In [ ]:
video_writer = cv2.VideoWriter(os.path.join('Output', 'mtcnn_face_edges_'+video_name), cv2.VideoWriter_fourcc(*'DIVX'), fps, (300, 300), isColor=False)

video_edges = [cv2.Canny(face_frame, threshold1=30, threshold2=40) for face_frame in video_frames]
[video_writer.write(face_edge) for face_edge in video_edges]

cap.release()
video_writer.release()

f, axarr = plt.subplots(1,2)
axarr[0].imshow(video_frames[0])
axarr[1].imshow(video_edges[0])
axarr[0].axis('off')
axarr[1].axis('off')
plt.plot()

# 3.2 - Face Crop MTCNN + Canny Edge Detector + Gaussian Blur.

In [ ]:
video_writer = cv2.VideoWriter(os.path.join('Output', 'mtcnn_face_edges_gaublur_'+video_name), cv2.VideoWriter_fourcc(*'DIVX'), fps, (300, 300), isColor=False)

blur_video = [cv2.GaussianBlur(face_frame, (5,5), 0) for face_frame in video_frames]
video_edges = [cv2.Canny(face_frame, threshold1=30, threshold2=40) for face_frame in blur_video]
[video_writer.write(face_edge) for face_edge in video_edges]

cap.release()
video_writer.release()

f, axarr = plt.subplots(1,2)
axarr[0].imshow(video_frames[0])
axarr[1].imshow(video_edges[0])
axarr[0].axis('off')
axarr[1].axis('off')
plt.plot()

# 3.2 - Face Crop MTCNN + Canny Edge Detector + Gaussian Blur + Gray Scale.

In [ ]:
video_writer = cv2.VideoWriter(os.path.join('Output', 'mtcnn_face_edges_gaublur_gray'+video_name), cv2.VideoWriter_fourcc(*'DIVX'), fps, (300, 300), isColor=False)

grey_video = [cv2.cvtColor(face_frame, cv2.COLOR_BGR2GRAY) for face_frame in video_frames]
blur_video = [cv2.GaussianBlur(face_frame, (5,5), 0) for face_frame in video_frames]
video_edges = [cv2.Canny(face_frame, threshold1=30, threshold2=40) for face_frame in blur_video]

[video_writer.write(face_edge) for face_edge in video_edges]
cap.release()
video_writer.release()

f, axarr = plt.subplots(1,2)
axarr[0].imshow(video_frames[0])
axarr[1].imshow(video_edges[0])
axarr[0].axis('off')
axarr[1].axis('off')
plt.plot()

# 4- SIFT Detection

Applying SIFT Detection to identify points of interest of the image

In [ ]:
def SIFT(img):
  sift_detector = cv2.SIFT_create(nfeatures=2000)
  final_img = img.copy()
  keypoints = sift_detector.detect(final_img)
  final_img = cv2.drawKeypoints(final_img, keypoints, final_img, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

  return final_img


video_SIFT = [SIFT(face_frame) for face_frame in video_frames.copy()]

f, axarr = plt.subplots(1,2)
axarr[0].imshow(video_frames[0])
axarr[1].imshow(video_edges[0])
axarr[0].axis('off')
axarr[1].axis('off')
plt.plot()


# 5- Color Histogram

In [ ]:
from collections import Counter

In [ ]:
def create_color_hist(img):
  channels = [Counter(img[:,:,i].reshape(-1)) for i in range(3)]
  hist = {i:[channels[i][intensity] for intensity in range(256)] for i in range(3)}

  return hist

plt.plot(create_color_hist(video_frames[0])[0], color='red')
plt.plot(create_color_hist(video_frames[0])[1], color='green')
plt.plot(create_color_hist(video_frames[0])[2], color='blue')
plt.show()


In [ ]:
prev = create_color_hist(video_frames[1])
next = create_color_hist(video_frames[2])

diff = next.copy()
# print(diff)
for key in prev:
  diff[key] = np.array(next[key]) - np.array(prev[key])

In [ ]:
plt.plot(diff[0], color='red')
plt.plot(diff[1], color='green')
plt.plot(diff[2], color='blue')
plt.show()